# Notebook for testing

In [1]:
from article_parser import get_covid_counts

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Gab Daos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
get_covid_counts()

C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\article_parser.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  df = pd.concat(dfs,ignore_index=True)


PermissionError: [Errno 13] Permission denied: 'ncov_parsed.csv'

In [11]:
df[df['counts'] == '1300\n\nConfirmed']

,author,category,date,source_id,source_name,text,title,LDA_Topics,count_docs,PH_Loc,status,counts,case,Loc


In [15]:
import pandas as pd
import numpy as np
from scipy import stats 
import geopandas as gpd
import datetime

df = pd.read_csv('pre_processed_data.csv')

# Filtering tables to those with PH Locations identified -----------------------

df = df[df['counts'] != '1300\r\n\r\nConfirmed']
df = df[df['Loc'] != '']



df['counts'] = df.counts.astype(int)
df = df.sort_values('date')

# Parse the results ------------------------------------------------------
def parse(df):
    #print(df.info())

    # Get min/max/mean values
    dfa = pd.pivot_table(df, values = 'counts', index=['date', 'Loc'], columns='status', aggfunc=[min, max, np.mean, stats.mode])

    # Remove multi-index
    dfa.columns = ["_".join(pair) for pair in dfa.columns]
    dfa = dfa.reset_index()

    # Replace 0 with np.nan to forward fill null values
    dfa = dfa.replace(0, np.nan)

    # Forward filling needs to be by area
    places = list(df['Loc'].unique())

    global dfb
    dfb = pd.DataFrame()
    for place in places:
        df_temp = dfa[dfa['Loc'] == place].fillna(method='ffill')
        dfb = dfb.append(df_temp)
    return dfb

res = parse(df)

# Cleaning results -----------------------------------------------------

res = res[['date','Loc', 'min_suspected','min_confirmed']]
res = res.fillna(0)
res = res.sort_values('date')

# Completing running total per date -------------------------------------

def add_row(df, row):
    df.loc[-1] = row
    df.index = df.index + 1  
    return df.sort_index()

for Loc in res['Loc']:
    sus_holder = 0
    con_holder = 0
    for date in res['date']:
        if sum(res[res['date'] == date]['Loc'].str.contains(Loc)) > 0:

            sus_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_suspected'].iloc[0]
            con_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_confirmed'].iloc[0]
        else:
            add_row(res, [date, Loc, sus_holder, con_holder])


res = res.sort_values('date')

# Including the geolocations of each location -------------------------------------

prov = gpd.read_file('prov_shp/prov_geo.shp')
df = pd.merge(res, prov, left_on = 'Loc', right_on = 'Pro_Name')

df['lat'] = [cor.split(',')[0] for cor in df['centroid']]
df['long'] = [cor.split(',')[1] for cor in df['centroid']]

# Cleaning final results -------------------------------------------------------

df = df[['date','Loc','min_suspected','min_confirmed','long','lat']]
df.columns = (['Date','Location','Suspected','Confirmed','Longitude','Latitude'])

df['Date'] = [datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime('%Y-%m-%dT%H:%M:%S.%f') for date in df['Date']]

df = df.sort_values('Date')

df.to_csv('ncov_parsed.csv', index = False)


SyntaxError: 'return' outside function (<ipython-input-15-379a12db214a>, line 90)

In [3]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.request import Request, urlopen

In [27]:
def get_mt():    
    user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0)'
    page_url = 'https://www.manilatimes.net/?s=coronavirus'
    manila_times_news = []

    df_list = pd.read_csv('articles_list/m_times_articles_urls.csv', header = None);
    article_list = df_list[0]

    while page_url != False:
        req = Request(page_url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'})
        content = urlopen(req).read()
        soup = BeautifulSoup(content)
        res = (soup.find_all('a',{'rel':"bookmark"}))

        #manila_times_news += [url['href'] for url in res]
        for url in res:
            if url['href'] in article_list.values:
                page_url = False
                print(url['href'] + ' stopped latest')
                break
            else:
                manila_times_news.append(url['href'])

        if page_url == False:
            break

        date = (soup.find_all('time'))

        page_link = soup.find('div',{'class':"page-nav"} )

        time.sleep(3)
        if date[0]['datetime'] < '2020-01-20':
            page_url = False
        else:
            page_url =  page_link.find_all('a')[-1]['href']
            print(page_url)

    mt_df = pd.DataFrame(columns = ['source_id','date','category','title','author','text'])
    df = pd.read_csv('scraped_data/manila_times_scraped.csv')

    try:    
        for article in manila_times_news:  
            user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
            req = Request(article, headers={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})
            content = urlopen(req).read()
            soup = BeautifulSoup(content)

            try:
                article_id = ''

                date = soup.find('time').text

                category = 'nCov'
                title = soup.find('h1', {'class' : 'tdb-title-text'}).text
                author = soup.find('a', {'class' : 'tdb-author-name'}).text

                text = soup.find('div', {'class' : 'td-post-content'})
                text.find_all('p')

                paragraph = ""
                for x in text.find_all('p'):
                    paragraph += x.text.strip() + " " 

                text = paragraph
            except AttributeError:
                continue

            if (title in df['title'].values):
                print('Reached latest article')
                break

            print(title, date)
            mt_df = mt_df.append(pd.Series([article_id,date,category, title,author, text], index = mt_df.columns ), ignore_index=True)
            time.sleep(2)

        df = df.append(mt_df)
        df.to_csv('scraped_data/manila_times_scraped.csv', index = False)

        df_list[0].append(manila_times_news).reset_index(drop = True).to_csv('articles_list/m_times_articles_urls.csv', index = False)

    except:
        df = df.append(mt_df)
        df.to_csv('scraped_data/manila_times_scraped.csv', index = False)


In [28]:
get_mt()

https://www.manilatimes.net/2020/03/15/news/latest-stories/ferrari-suspends-production-over-coronavirus/703354/ stopped latest


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:89: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [6]:
pd.Series(manila_times_news).to_csv('articles_list/m_times_articles_url.csv', index = False)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [21]:
df_list = pd.read_csv('articles_list/m_times_articles_url.csv', header = None);

In [22]:
articles_list = df_list[0]

In [23]:
mt_df = pd.DataFrame(columns = ['source_id','date','category','title','author','text'])
df = pd.read_csv('scraped_data/manila_times_scraped.csv')
try:    
    for article in articles_list:  
        user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
        req = Request(article, headers={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'})
        content = urlopen(req).read()
        soup = BeautifulSoup(content)

        try:
            article_id = ''

            date = soup.find('time').text

            category = 'nCov'
            title = soup.find('h1', {'class' : 'tdb-title-text'}).text
            author = soup.find('a', {'class' : 'tdb-author-name'}).text

            text = soup.find('div', {'class' : 'td-post-content'})
            text.find_all('p')

            paragraph = ""
            for x in text.find_all('p'):
                paragraph += x.text.strip() + " " 

            text = paragraph
        except AttributeError:
            continue

        if (title in df['title'].values):
            print('Reached latest article')
            break

        print(title, date)
        mt_df = mt_df.append(pd.Series([article_id,date,category, title,author, text], index = mt_df.columns ), ignore_index=True)
        time.sleep(2)

    df = df.append(mt_df)
    df.to_csv('scraped_data/new_manila_times_scraped.csv', index = False)
except:
    df = df.append(mt_df)
    df.to_csv('scraped_data/new_manila_times_scraped.csv', index = False)

Ferrari suspends production over coronavirus March 15, 2020
British football shuts down to halt coronavirus spread March 15, 2020
Raptors test negative for coronavirus March 15, 2020
Augusta National postpones Masters golf due to coronavirus March 15, 2020
Coronavirus world: Science and math key to managing March 15, 2020
Supply and demand, hoarding, price gouging — and the coronavirus March 15, 2020
India and the coronavirus: Lucky escape or emergency ahead? March 15, 2020
Joining Hands in Coronavirus Combat: Policy Recommendations March 14, 2020
Filipino diplomat at UN coronavirus-infected March 14, 2020
Coronavirus-infected people forbidden to enter mosques March 14, 2020
NBA coronavirus shutdown likely to be ‘at least 30 days’ – Silver March 14, 2020
Bong Go backs Congress special session to OK Coronavirus budget March 14, 2020
LRT-1 maintains normal operating schedule March 15, 2020
Spain imposes near total lockdown to fight virus March 15, 2020
Ferrari suspends production over co

In [70]:
mt_df_holder = mt_df

In [71]:
mt_df.to_csv('scraped_data/manila_times_scraped.csv', index = False)

In [ ]:
user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
req = Request(article, headers={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
'Accept-Encoding': 'none',
'Accept-Language': 'en-US,en;q=0.8',
'Connection': 'keep-alive'})
content = urlopen(req).read()
soup = BeautifulSoup(content)

article_id = ''

date = soup.find('time').text

category = 'nCov'
title = soup.find('h1', {'class' : 'tdb-title-text'}).text
author = soup.find('a', {'class' : 'tdb-author-name'}).text

text = soup.find('div', {'class' : 'td-post-content'})
text.find_all('p')

paragraph = ""
for x in text.find_all('p'):
    paragraph += x.text.strip() + " " 

text = paragraph


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('scraped_data/abscbn_scraped.csv')

In [13]:
test = 'Australia confirms 1st coronavirucases'

In [14]:
article_list = list(df['title'].unique())

In [15]:
test in article_list

False

In [1]:
from news_scrapers import get_mb

In [2]:
get_mb()

C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\news_scrapers.py:175: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 175 of the file C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\news_scrapers.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(content)


start
https://news.mb.com.ph/tag/ncov/page1/
https://news.mb.com.ph/2020/03/15/medical-practitioner-is-2nd-confirmed-covid-19-case-in-cavite/ stopped latest
0


C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\news_scrapers.py:199: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 199 of the file C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\news_scrapers.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(content)
C:\Users\Gab Daos\Documents\Mago\Git Projects\COVID19TrackerPH\gab\news_scrapers.py:262: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  df_list[0].append(to_add).